In [2]:
!pip install ipython-autotime

  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [1]:
import cudf
import nvcategory
import nvtext
import nvstrings

%load_ext autotime

In [9]:
df = cudf.read_csv(
    '../data/parking_Jun2019.csv'
)

time: 3.55 s


In [8]:
df.dtypes

OccupancyDateTime           datetime64[ms]
PaidOccupancy                        int64
BlockfaceName                       object
SideOfStreet                        object
SourceElementKey                     int64
ParkingTimeLimitCategory             int64
ParkingSpaceCount                    int64
PaidParkingArea                     object
PaidParkingSubArea                  object
PaidParkingRate                       int8
ParkingCategory                     object
Location                            object
dtype: object

time: 7.58 ms


In [11]:
print(df.columns)

Index(['OccupancyDateTime', 'PaidOccupancy', 'BlockfaceName', 'SideOfStreet',
       'SourceElementKey', 'ParkingTimeLimitCategory', 'ParkingSpaceCount',
       'PaidParkingArea', 'PaidParkingSubArea', 'PaidParkingRate',
       'ParkingCategory', 'Location'],
      dtype='object')
time: 990 µs


In [32]:
df['hour'] = df['OccupancyDateTime']._column.hour
df['minute'] = df['OccupancyDateTime']._column.minute

print(df[['OccupancyDateTime','hour', 'minute']])

         OccupancyDateTime  hour  minute
0 2019-06-10T11:00:00.000    11       0
1 2019-06-10T09:25:00.000     9      25
2 2019-06-10T19:37:00.000    19      37
3 2019-06-10T08:18:00.000     8      18
4 2019-06-10T10:30:00.000    10      30
5 2019-06-10T14:24:00.000    14      24
6 2019-06-10T15:16:00.000    15      16
7 2019-06-10T11:56:00.000    11      56
8 2019-06-10T14:51:00.000    14      51
9 2019-06-10T13:24:00.000    13      24
[23749262 more rows]
time: 116 ms


In [33]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0].str.slice(0,10)
    return lon.str.stod()

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1].str.slice(0,8)
    return lon.str.stod()
    
df['LON'] = extractLon(df['Location'])
df['LAT'] = extractLat(df['Location'])

print(df[['Location', 'LON', 'LAT']])

                             Location                 LON                 LAT
0   POINT (-122.2849609 47.55747782) -122.28495999999998  47.557469999999995
1  POINT (-122.33419318 47.61911853) -122.33418999999999            47.61911
2  POINT (-122.32064833 47.60984775) -122.32063999999998            47.60984
3  POINT (-122.31826395 47.61468863)          -122.31826   47.61467999999999
4  POINT (-122.38355354 47.66773569) -122.38354999999999            47.66773
5  POINT (-122.31750336 47.61402223)           -122.3175            47.61402
6  POINT (-122.33143883 47.60044775) -122.33142999999998            47.60044
7  POINT (-122.35666155 47.62035414) -122.35665999999999  47.620349999999995
8  POINT (-122.32281958 47.59757726) -122.32280999999999            47.59757
9  POINT (-122.32089936 47.61467932) -122.32088999999999            47.61467
[23749262 more rows]
time: 4.88 s


In [34]:
!head ../data/parking_Jun2019.csv

OccupancyDateTime,PaidOccupancy,BlockfaceName,SideOfStreet,SourceElementKey,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,PaidParkingRate,ParkingCategory,Location
2019-06-10T11:00:00,3,RAINIER AVE S BETWEEN S FERDINAND ST AND S HUDSON ST,W,12401,120,4,Columbia City,,,Paid Parking,POINT (-122.2849609 47.55747782)
2019-06-10T09:25:00,1,FAIRVIEW AVE N BETWEEN DENNY WAY AND JOHN ST,E,33142,120,3,South Lake Union,South,,Paid Parking,POINT (-122.33419318 47.61911853)
2019-06-10T19:37:00,1,BROADWAY BETWEEN BOYLSTON AVE AND MARION ST,E,32258,120,3,First Hill,,,Paid Parking,POINT (-122.32064833 47.60984775)
2019-06-10T08:18:00,3,11TH AVE BETWEEN E PIKE ST AND E PINE ST,W,24405,120,11,Pike-Pine,,,Paid Parking,POINT (-122.31826395 47.61468863)
2019-06-10T10:30:00,2,LEARY AVE NW BETWEEN NW VERNON PL AND NW MARKET ST,SW,56653,120,11,Ballard,Core,,Paid Parking,POINT (-122.38355354 47.66773569)
2019-06-10T14:24:00,2,E PIKE ST BETWEEN 11TH AVE AND 12TH AVE,S,14678,120,5

In [37]:
!pip install geopy

     |################################| 102kB 9.8MB/s ta 0:00:011
  Stored in directory: /root/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
time: 1.88 s


In [38]:
from geopy.geocoders import Nominatim

geolo

time: 13.4 ms


In [91]:
points = [
    'POINT (-122.33644763 47.61672619)'
    , 'POINT (-122.34481093 47.61765602)'
    , 'POINT (-122.34293648 47.61700881)'
    , 'POINT (-122.35812857 47.61944097)'
    , 'POINT (-122.33241002 47.60201883)'
    , 'POINT (-122.32628038 47.59793434)'
    , 'POINT (-122.32320088 47.60811139)'
    , 'POINT (-122.34613913 47.6203126)'
    , 'POINT (-122.32487277 47.60654852)'
    , 'POINT (-122.34746087 47.61501419)'
]

ids = range(len(points))


df_small = cudf.DataFrame({'ids': ids, 'points': points})
print(df_small)

locs_small = nvstrings.to_device(df_small['points'].to_pandas().tolist())
locs_conv = locs_small.extract_record('([0-9\.\-]+) ([0-9\.]+)')
lon, lat = zip(*[(float(e.to_host()[0]), float(e.to_host()[1])) for e in locs_conv])
df_latLon = cudf.DataFrame({'lon': lon, 'lat': lat})

    ids                             points
0    0  POINT (-122.33644763 47.61672619)
1    1  POINT (-122.34481093 47.61765602)
2    2  POINT (-122.34293648 47.61700881)
3    3  POINT (-122.35812857 47.61944097)
4    4  POINT (-122.33241002 47.60201883)
5    5  POINT (-122.32628038 47.59793434)
6    6  POINT (-122.32320088 47.60811139)
7    7   POINT (-122.34613913 47.6203126)
8    8  POINT (-122.32487277 47.60654852)
9    9  POINT (-122.34746087 47.61501419)


In [92]:
df_small.add_column('lat', df_latLon['lat'])
df_small.add_column('lon', df_latLon['lon'])
print(df_small)

    ids                             points          lat           lon
0    0  POINT (-122.33644763 47.61672619)  47.61672619 -122.33644763
1    1  POINT (-122.34481093 47.61765602)  47.61765602 -122.34481093
2    2  POINT (-122.34293648 47.61700881)  47.61700881 -122.34293648
3    3  POINT (-122.35812857 47.61944097)  47.61944097 -122.35812857
4    4  POINT (-122.33241002 47.60201883)  47.60201883 -122.33241002
5    5  POINT (-122.32628038 47.59793434)  47.59793434 -122.32628038
6    6  POINT (-122.32320088 47.60811139)  47.60811139 -122.32320088
7    7   POINT (-122.34613913 47.6203126)   47.6203126 -122.34613913
8    8  POINT (-122.32487277 47.60654852)  47.60654852 -122.32487277
9    9  POINT (-122.34746087 47.61501419)  47.61501419 -122.34746087


In [68]:
import nvstrings
s = nvstrings.to_device(
['POINT (-122.33644763 47.61672619)', 'POINT (-122.34481093 47.61765602)', 'POINT (-122.34293648 47.61700881)', 'POINT (-122.35812857 47.61944097)', 'POINT (-122.33241002 47.60201883)', 'POINT (-122.32628038 47.59793434)', 'POINT (-122.32320088 47.60811139)', 'POINT (-122.34613913 47.6203126)', 'POINT (-122.32487277 47.60654852)', 'POINT (-122.34746087 47.61501419)'])

for result in s.extract_record("([0-9\.\-]+) ([0-9\.]+)"):
    print(result)
# for result in s.extract_record('(POINT\()([0-9\.-]+) ([0-9\.-]+)(\))'):
#     print(result)

['-122.33644763', '47.61672619']
['-122.34481093', '47.61765602']
['-122.34293648', '47.61700881']
['-122.35812857', '47.61944097']
['-122.33241002', '47.60201883']
['-122.32628038', '47.59793434']
['-122.32320088', '47.60811139']
['-122.34613913', '47.6203126']
['-122.32487277', '47.60654852']
['-122.34746087', '47.61501419']


In [64]:
f = gpu_location.extract_record('([0-9\.\-]+) ([0-9\.]+)')
# print(f[1].gather([0,100]))

In [67]:
df.add_column('lon', [e[0] for e in f])

TypeError: Could not convert object to sequence

In [ ]:
df.add_column('lat', [e[1] for e in f])
print(df.head(10))

In [63]:
print(gpu_location.gather([1,2,3,4,5]))

['POINT (-122.34481093 47.61765602)', 'POINT (-122.34293648 47.61700881)', 'POINT (-122.35812857 47.61944097)', 'POINT (-122.33241002 47.60201883)', 'POINT (-122.32628038 47.59793434)']


In [11]:
Location = df['Location']

def extractLon(Location, location_lon):
    for i, point in enumerate(Location):
        ppp_split = point.split(' ')[1:]
        location_lon[i] = float(ppp_split[0][1:])

df.apply_rows(extractLon, incols=['Location'], outcols=dict(location_lon=np.float32), kwargs={})

AttributeError: 'nvstrings' object has no attribute 'to_gpu_array'

In [1]:
import nvstrings, nvcategory
import requests

url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

#split content into a list, remove header
host_lines = content.strip().split('\n')[1:]
host_lines

['16.99,1.01,Female,No,Sun,Dinner,2',
 '10.34,1.66,Male,No,Sun,Dinner,3',
 '21.01,3.5,Male,No,Sun,Dinner,3',
 '23.68,3.31,Male,No,Sun,Dinner,2',
 '24.59,3.61,Female,No,Sun,Dinner,4',
 '25.29,4.71,Male,No,Sun,Dinner,4',
 '8.77,2.0,Male,No,Sun,Dinner,2',
 '26.88,3.12,Male,No,Sun,Dinner,4',
 '15.04,1.96,Male,No,Sun,Dinner,2',
 '14.78,3.23,Male,No,Sun,Dinner,2',
 '10.27,1.71,Male,No,Sun,Dinner,2',
 '35.26,5.0,Female,No,Sun,Dinner,4',
 '15.42,1.57,Male,No,Sun,Dinner,2',
 '18.43,3.0,Male,No,Sun,Dinner,4',
 '14.83,3.02,Female,No,Sun,Dinner,2',
 '21.58,3.92,Male,No,Sun,Dinner,2',
 '10.33,1.67,Female,No,Sun,Dinner,3',
 '16.29,3.71,Male,No,Sun,Dinner,3',
 '16.97,3.5,Female,No,Sun,Dinner,3',
 '20.65,3.35,Male,No,Sat,Dinner,3',
 '17.92,4.08,Male,No,Sat,Dinner,2',
 '20.29,2.75,Female,No,Sat,Dinner,2',
 '15.77,2.23,Female,No,Sat,Dinner,2',
 '39.42,7.58,Male,No,Sat,Dinner,4',
 '19.82,3.18,Male,No,Sat,Dinner,2',
 '17.81,2.34,Male,No,Sat,Dinner,4',
 '13.37,2.0,Male,No,Sat,Dinner,2',
 '12.69,2.0,Male,No

In [27]:
#copy strings to gpu
gpu_lines = nvstrings.to_device(host_lines)

#split into columns on gpu
gpu_columns = gpu_lines.split(',')
gpu_day_of_week = gpu_columns[4]

#use gpu `replace` to re-encode tokens on GPU
for idx, day in enumerate(['Sun', 'Mon', 'Tues', 'Wed', 'Thur', 'Fri', 'Sat']):
    gpu_day_of_week = gpu_day_of_week.replace(day, str(idx))

# or, use nvcategory's builtin GPU categorization
cat = nvcategory.from_strings(gpu_columns[4])

# copy category keys to host and print
print(cat.keys())

# copy "cleaned" strings to host and print
print(gpu_day_of_week)

['Fri', 'Sat', 'Sun', 'Thur']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '0', '0', '0', '0', '0', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '6', '6', '6', '6', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '4', '4', '4',

In [29]:
print(gpu_lines)

['16.99,1.01,Female,No,Sun,Dinner,2', '10.34,1.66,Male,No,Sun,Dinner,3', '21.01,3.5,Male,No,Sun,Dinner,3', '23.68,3.31,Male,No,Sun,Dinner,2', '24.59,3.61,Female,No,Sun,Dinner,4', '25.29,4.71,Male,No,Sun,Dinner,4', '8.77,2.0,Male,No,Sun,Dinner,2', '26.88,3.12,Male,No,Sun,Dinner,4', '15.04,1.96,Male,No,Sun,Dinner,2', '14.78,3.23,Male,No,Sun,Dinner,2', '10.27,1.71,Male,No,Sun,Dinner,2', '35.26,5.0,Female,No,Sun,Dinner,4', '15.42,1.57,Male,No,Sun,Dinner,2', '18.43,3.0,Male,No,Sun,Dinner,4', '14.83,3.02,Female,No,Sun,Dinner,2', '21.58,3.92,Male,No,Sun,Dinner,2', '10.33,1.67,Female,No,Sun,Dinner,3', '16.29,3.71,Male,No,Sun,Dinner,3', '16.97,3.5,Female,No,Sun,Dinner,3', '20.65,3.35,Male,No,Sat,Dinner,3', '17.92,4.08,Male,No,Sat,Dinner,2', '20.29,2.75,Female,No,Sat,Dinner,2', '15.77,2.23,Female,No,Sat,Dinner,2', '39.42,7.58,Male,No,Sat,Dinner,4', '19.82,3.18,Male,No,Sat,Dinner,2', '17.81,2.34,Male,No,Sat,Dinner,4', '13.37,2.0,Male,No,Sat,Dinner,2', '12.69,2.0,Male,No,Sat,Dinner,2', '21.7,4.3,M